In [107]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import ConfigurableField
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_ollama.llms import OllamaLLM
from langchain_ollama import ChatOllama
from typing import Tuple, List, Optional
from neo4j import GraphDatabase
from langchain.chains import GraphCypherQAChain
from langchain.embeddings import HuggingFaceEmbeddings
# from pydantic.v1 import BaseModel, Field
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

from neo4j import GraphDatabase
from langchain_community.vectorstores import Neo4jVector
from langchain_community.graphs import Neo4jGraph
from langchain.vectorstores import FAISS

import pickle
import os

In [3]:
NEO4J_URI="neo4j+s://9269fc71.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="lOnpSSiAFQg0uWzA1EvYYBhK45_rWbxXb9SHRDwZRfk"

In [4]:
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [5]:
graph = Neo4jGraph()

In [6]:
graph_documents = Neo4jGraph(url=os.environ["NEO4J_URI"], username=os.environ["NEO4J_USERNAME"], password=os.environ["NEO4J_PASSWORD"])

In [85]:
llm = ChatOllama(model='llama3.2', temperature=0, num_gpu=torch.cuda.device_count())

In [7]:
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name,
  model_kwargs=model_kwargs
)

C:\Users\chand\AppData\Local\Temp\ipykernel_2416\576819256.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
C:\Users\chand\.conda\envs\fusgraph\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={'device': 'cuda'}, encode_kwargs={}, multi_process=False, show_progress=False)

In [10]:
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [17]:
vector_index

In [71]:
allowed_nodes = [
    "Disease", "Cause", "Symptom", "Complication", "Treatment", "Medication",
    "Test", "Risk Factor", "Diagnosis", "Side Effect", "Procedure", 
    "Condition Stage", "Disease Type", "Imaging Type"
]

In [88]:
# Define the schema without validators to avoid deepcopy issues
class Entities(BaseModel):
    """Identifying information about entities."""

    nodes: List[str] = Field(
        ...,
        description="All the Disease, Cause, Symptom, Complication, Treatment, Medication, Test, Risk Factor, Diagnosis, Side Effect, Procedure, Condition Stage, Disease Type, Imaging Type"
                    "that appear in the text"
    )

In [149]:
# Define the prompt template with instructions for node extraction
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are tasked with extracting the diseases, conditions, Diagnosis or symptoms from the qiven question "
            " for example, for the question: How is a gastric ulcer diagnosed?, you are going to find that gastric ulcer is a dieases and you also found that diagnosis is used in the question, so you are going to return gastric ulcer"
            # "You are tasked with extracting Disease, Cause, Symptom, Complication, Treatment, Medication, Test, Risk Factor, Diagnosis, Side Effect, Procedure, Condition Stage, Disease Type, Imaging Type from the text"
        ),
        (
            "human",
            "Use the specified format to extract information from the following input: {question}."
        ),
    ]
)

In [150]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [151]:
entity_chain.invoke({"question": "How do NSAIDs induce gastric mucosal injury leading to ulcers?"})

Entities(nodes=['Gastric ulcer', 'NSAID-induced gastric mucosal injury'])

In [152]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

In [153]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.nodes:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query)
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [165]:
print(structured_retriever("How do NSAIDs induce gastric mucosal injury leading to ulcers?"))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 93} for query: "CALL db.index.fulltext.queryNodes('entity', $query)\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.Fe

Gastric ulcer - ASSOCIATED_WITH -> Bleeding
Gastric ulcer - ASSOCIATED_WITH -> Peptic ulcer disease
Gastric ulcer - ASSOCIATED_WITH -> Mass
Gastric ulcer - HAS_SYMPTOM -> Vomiting
Gastric ulcer - HAS_SYMPTOM -> Nausea
Gastric ulcer - HAS_SYMPTOM -> Pain soon after meals
Gastric ulcer - HAS_SYMPTOM -> Weight loss
Gastric ulcer - HAS_SYMPTOM -> Anorexia
H2RAs - TREATED_WITH -> Gastric ulcer
PPIs - TREATED_WITH -> Gastric ulcer
Sucralfate - TREATED_WITH -> Gastric ulcer
Misoprostol - TREATED_WITH -> Gastric ulcer
Proton pump inhibitors - TREATED_WITH -> Gastric ulcer
P-CAB - TREATED_WITH -> Gastric ulcer
Pirenzepine - TREATED_WITH -> Gastric ulcer
proton pump inhibitors - TREATED_WITH -> gastric ulcer
Gastric Ulcer - HAS_SYMPTOM -> ulcer stenosis; pyloric stenosis; oesophageal gastric varices; Barrett's oesophagus measuring >3 cm; intractable ulcer; digestive ulcer perforation or malignancy on upper GI endoscopy
Gastric Ulcer - HAS_SYMPTOM -> GI symptoms
Gastric Ulcer - COMPLICATES -> adv

In [155]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [156]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [157]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [158]:
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [159]:
_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOllama(model='llama3.2', temperature=0, num_gpu=torch.cuda.device_count())
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [160]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""

In [161]:
prompt = ChatPromptTemplate.from_template(template)

In [162]:
chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [163]:
result = chain.invoke({"question": "How do NSAIDs induce gastric mucosal injury leading to ulcers?"})

Search query: How do NSAIDs induce gastric mucosal injury leading to ulcers?


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 93} for query: "CALL db.index.fulltext.queryNodes('entity', $query)\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.Fe

In [164]:
print(result)

NSAIDs induce gastric mucosal injury by suppressing prostaglandin synthesis, which reduces mucosal defense. This leads to neutrophil adherence, oxygen-free radical release, protease activation, and capillary blood flow obstruction. Additionally, nitric oxide (NO) and hydrogen sulfide (H2S) play a role in maintaining the gastric mucosal barrier by increasing blood flow, stimulating mucus secretion, and inhibiting neutrophil adherence.
